In [1]:
import networkx as nx
from types_definition import * 
from vector import Vector
from rotorconfig import RotorConfig
from rotorgraph import RotorGraph, display_path, all_config_from_recurrent, display_grid
from particleconfig import ParticleConfig
from matrices import Matrix
from arcsum import ArcSum
from arcmonic import Arcmonic
import numpy as np
from smithnormalform import matrix, snfproblem, z
from otherfunctions import *
import random




Algo de résolution de arrival dans le cas des path muligraphs avec x et y non premiers entre eux.

cas Y>X

In [22]:
#Creation du graphe

m=4

n=5
x=2
y=3

rotor_graph = RotorGraph.simple_path(n,m*y,m*x)




L= rotor_graph.laplacian_matrix()
rL=rotor_graph.reduced_laplacian_matrix()

#print(L, "\n\n", rL)


prob = L.snf_problem()


MatrixT = Matrix(prob.T)
MatrixT = MatrixT.to_numpy()

MatrixJ = Matrix(prob.J)
MatrixJ = MatrixJ.to_numpy()

print(MatrixJ)
print()
print(MatrixT)

#print(rotor_graph.edges)

[[4 0 0 0 0 0 0]
 [0 4 0 0 0 0 0]
 [0 0 4 0 0 0 0]
 [0 0 0 4 0 0 0]
 [0 0 0 0 4 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]

[[  -9  -32 -105 -332   -2 -665 -664]
 [  -7  -28  -97 -316   -1 -633 -632]
 [  -4  -22  -85 -292    0 -585 -584]
 [   0  -13  -67 -256    0 -513 -512]
 [   0    0  -40 -202    0 -405 -404]
 [   0    0    0 -121    0 -243 -242]
 [   0    0    0    0    0    0    1]]


In [23]:
import random
sigma = ParticleConfig(rotor_graph)
rho = RotorConfig(rotor_graph)

#config de rotor au hazard
for i in range(1,n+1):
    eh= random.randint(0, len(rotor_graph.rotor_order[i])-1)
    rho[i]= rotor_graph.rotor_order[i][eh]

#config de particule au hazard
for i in range(1, n+1):
    sigma[i] = random.randint(0, 20) 

display_path(rho, sigma)
print(rho)

#routage aux puits
sigma_f, rho, info = rotor_graph.legal_routing(sigma, rho)
#display_path(rho, sigma)

#print(sigma)
#print(sigma_f)
#print(sigma + sigma_f)


0 - 18 - 12 - 9 - 15 - 15 - 0
{1: (1, 0, 11), 2: (2, 1, 10), 3: (3, 2, 9), 4: (4, 3, 4), 5: (5, 4, 8)}


In [24]:
import numpy as np
#liste des diff:

diff = []

for i in range(n+1):
    diff.append([])
    for j in range(n+1):
        if j==i:
            diff[i].append(1)
        else:
            diff[i].append(0)
    diff[i].append((x**(n-i))*(y**i))

diff_array = [np.array(ligne) for ligne in diff]

print(diff_array)

#valeur harmonique d'un sommet
def h(sommet:int, n: int, diff_array):
    harmonic = np.array([0 for _ in range(n+2)])

    for i in range(sommet):
        harmonic += diff_array[i]
    return harmonic

#ordre du groupe
F= h(n+1,n,diff_array)[-1]

#valeur harmonique d'une config sigma
def h_sigma(sigma, n=n, diff_array=diff_array):
    h_sigma = 0
    for i in range(len(sigma)):
        h_sigma += sigma[i]*h(i,n,diff_array)
    return h_sigma

#affichage d'un calcul
print(h_sigma(sigma))
print(sigma)

#valeur arcmonique d'une config
def a_rho(rho, n=n, diff_array=diff_array):
    a_sigma = np.array([0 for _ in range(n+2)])
    for i in range(1,n+1):
        edge = rho[i]
        if edge[0]< edge[1]:
            a_sigma += edge[2]*diff_array[i]
        else:
            a_sigma += (m*y-edge[2])*diff_array[i-1] #

    return a_sigma

##affichage d'un calcul
#config de rotor au hazard
for i in range(1,n+1):
    eh= random.randint(0, len(rotor_graph.rotor_order[i])-1)
    rho[i]= rotor_graph.rotor_order[i][eh]
    

#print(a_rho(rho))
#print(rho)
print(F)


[array([ 1,  0,  0,  0,  0,  0, 32]), array([ 0,  1,  0,  0,  0,  0, 48]), array([ 0,  0,  1,  0,  0,  0, 72]), array([  0,   0,   0,   1,   0,   0, 108]), array([  0,   0,   0,   0,   1,   0, 162]), array([  0,   0,   0,   0,   0,   1, 243])]
[   69    51    39    30    15     0 13134]
{0: 0, 1: 18, 2: 12, 3: 9, 4: 15, 5: 15, 6: 0}
665


In [25]:
#Engel machine

Engel = RotorGraph()
for i in range(n+2): Engel.add_node(i)

for i in range(n):
    for _ in range(m*x):
        Engel.add_edge(i,i+1)
    if y>x:
        for _ in range(m*x,m*y):
            Engel.add_edge(i,n+1)
if not(y>x):
    print("Y doit etre inférieur à X")

'''
import matplotlib.pyplot as plt

nx.draw(Engel, with_labels=True, node_color='skyblue', node_size=1000, font_size=12, font_weight='bold', edge_color='gray')

# Afficher le dessin
plt.title("Multigraphe")
plt.show()

print(Engel.edges)
'''

Engel_Laplacian = Engel.laplacian_matrix()
Engel_L= Engel_Laplacian.to_numpy()
print(Engel_L)


[[12 -8  0  0  0  0 -4]
 [ 0 12 -8  0  0  0 -4]
 [ 0  0 12 -8  0  0 -4]
 [ 0  0  0 12 -8  0 -4]
 [ 0  0  0  0 12 -8 -4]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]]


In [26]:
#stabilisation with Engel Machine

vec = np.array([0 for _ in range(n+1)])
vec = np.array([4,-1,4,-1,0])

def stab_engel(vec,Engel=Engel):
    Engel_Laplacian = Engel.laplacian_matrix()
    Engel_L= Engel_Laplacian.to_numpy()

    sigma_engel = ParticleConfig(Engel)
    #initialisation
    for i in range(n+1):
        sigma_engel[i] = vec[i]
    
    for sommet in range(n):
        nb_tirage = (sigma_engel[sommet] - (sigma_engel[sommet])%(m*y))//(m*y)
        vec_tirage = nb_tirage * Engel_L[sommet]

        for j in range(n+2):
            sigma_engel[j]-=vec_tirage[j]

    return np.array(list(sigma_engel.values()))

#exemple de stabilisation.
#vec = np.array([-22,-15,-9,-4,0]) # +8* np.array([1 for _ in range(n+1)])
#print(stab_engel(vec))

#recherche de la valeur stabilisante

def val_stab_engel(vec,Engel=Engel):
    val=0
    stab = stab_engel(vec,Engel=Engel)
    while not  (0<=stab[n]<=m*x):
        val +=1
        stab = stab_engel(vec + val*np.array([1 for _ in range(n+1)]) ,Engel=Engel)
    return val, stab
#print(val_stab_engel(vec))




In [27]:
#gros exemple test de l'algo



import random
import math
import time



#config de rotor au hazard
rho = RotorConfig(rotor_graph)
for i in range(1,n+1):
    eh= random.randint(0, len(rotor_graph.rotor_order[i])-1)
    rho[i]= rotor_graph.rotor_order[i][eh]

#config de particule au hazard
sigma = ParticleConfig(rotor_graph)
for i in range(1, n+1):
    sigma[i] = random.randint(0, 10) 

###Routage normale avec une config de rotor et de particule aux hasards
#routage aux puits

t1 = time.time()
sigma_f, rho_f, info = rotor_graph.legal_routing(sigma, rho)
t2 = time.time()
print("temps routage =", t2-t1)
print(sigma)
print(sigma_f)


t3= time.time()
a_rho_f = a_rho(rho_f)
a_rho_ff=np.array([a_rho_f[i] for i in range(len(a_rho_f)-1)])

#print(val_stab_engel(a_rho_ff))

####Test avec l'algo


vect = a_rho(rho)-h_sigma(sigma)



vec = np.array([vect[i] for i in range(len(vect)-1)])

#accéléer le truc en rendant la dernière composante positive
if vect[-1] < 0:
    val = math.ceil(-vect[-1]/F)
    vec += val*np.array([1 for _ in range(n+1)])




b,c = val_stab_engel(vec)
t4= time.time()

b= b+val
print(val,b,c)

print("temps algo =", t4-t3)


temps routage = 0.0058934688568115234
{0: 0, 1: 7, 2: 5, 3: 10, 4: 8, 5: 5, 6: 0}
{0: 27, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 8}
7 8 [  6   3   1   7   1   0 -48]
temps algo = 0.0005853176116943359
